In [1]:
import numpy as np
import json
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output
from easydict import EasyDict as edict

In [8]:
DATASET_PATH = '/home/ivan/Documents/antares_mount/0_datasets/0_trainval_datasets/dji/20190905_json17_testing'
SAVE_PATH = '/home/ivan/Documents/6_projects/dist_cnn/dataset/augmented/raw'
RANDOM_CROP_SIZE = 64
BASE_CROP_SIZE = 96

In [4]:
def get_data(file_name):
    with open(txt_path) as f:
        raw_data = json.load(f)
        
    data = edict({})
    data.im_h = raw_data['Camera']['Height']
    data.im_w = raw_data['Camera']['Width']
    data.objects = []
    for obj in raw_data['Objects']:
        o = edict({})
        o.cls = cls = obj['Name']
        o.region = obj['Regions'][0]
        o.pos_x_3d = float(obj['Position']['x'])
        o.pos_y_3d = float(obj['Position']['y'])
        o.pos_z_3d = float(obj['Position']['z'])
        o.distance = get_dist((o.pos_x_3d, o.pos_y_3d, o.pos_z_3d))
        o.w = int(o.region['VisibleBox']['W'])
        o.h = int(o.region['VisibleBox']['H'])
        o.x_ctr = int(o.region['VisibleBox']['X'] + o.w / 2)
        o.y_ctr = int(data.im_h - 1 -(o.region['VisibleBox']['Y'] + o.h / 2))
        data.objects.append(o)
    return data

In [9]:
def get_crop_roi(x_ctr, y_ctr, w, h):
    mult = max(w, h) // 40 + 1  # 40 - depends on the base crop size
    crop_size = BASE_CROP_SIZE * mult
    
    left = int(x_ctr - crop_size / 2)
    right = int(x_ctr + crop_size / 2)
    top = int(y_ctr - crop_size / 2)
    bottom = int(y_ctr + crop_size / 2)
    return left, right, top, bottom, mult

In [10]:
def get_dist(point):
    return np.sqrt(np.square(point[0]) + np.square(point[1]) + np.square(point[2]))

In [11]:
def get_crops_from_image(img_path, txt_path, labels):
    img = cv2.imread(img_path)
    data = get_data(txt_path)
    obj_num = 0
    for o in data.objects:
        left, right, top, bottom, mult = get_crop_roi(o.x_ctr, o.y_ctr, o.w, o.h)
        
        if (left < 0) \
        or (top < 0) \
        or (right >= data.im_w - 1) \
        or (bottom >= data.im_h - 1):
            continue
            
        cropped_img = img[top:bottom, left:right, :]
        dir_name = img_path.split('/')[-2]
        image_name = img_path.split('/')[-1].split('.')
        image_name = '_'.join([dir_name, image_name[0], str(obj_num), '.' + image_name[1]])
        labels.append({'name': image_name, 'dist': o.distance, 'mult': mult})
        imgs_save_path = os.path.join(SAVE_PATH, image_name)
        cv2.imwrite(imgs_save_path, cropped_img)
        
        obj_num += 1

In [12]:
data = {'data': []}
 
for foldername in os.listdir(DATASET_PATH):
    folderpath = os.path.join(DATASET_PATH, foldername)
    file_num = 0
    folder_num = int(foldername[3])
    total_files = len(os.listdir(folderpath)) // 2
    
    for filename in os.listdir(folderpath):
        if filename.endswith('.png'):
            img_path = os.path.join(folderpath, filename)
            txt_path = os.path.join(folderpath, filename[:-8] + '.txt')
            
            get_crops_from_image(img_path, txt_path, data['data'])
                
            file_num += 1
            clear_output(wait=True)
            print(filename)
            print(f'{file_num}/{total_files},', file_num * 100 // total_files, '%', )
            
    print('Ready. Total images:', len(os.listdir(SAVE_PATH)))
labels_path = os.path.join(SAVE_PATH, 'labels.json')
with open(labels_path, 'w+') as labels:
    json.dump(data, labels)

88d732a18d6625ba_rgb.png
2000/2000, 100 %
Ready. Total images: 85710


img = cv2.imread('/home/ivan/Documents/6_projects/dist_cnn/dataset/lev1_88d731dd2c550936_rgb_0_.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.rcParams['figure.figsize'] = [10, 5]
plt.scatter([x['dist'] for x in data['data']],[x['mult'] for x in data['data']])

plt.hist([x['dist'] for x in data['data']], bins=20)

In [7]:
with open(os.path.join(SAVE_PATH, 'full', 'labels.json')) as labels:
    labels = json.load(labels)['data']

In [10]:
labels = [x for x in labels if x['mult'] == 1]
samples = np.random.choice(labels, 10)


array([{'name': 'lev4_88d73204f3e693c6_rgb_9_.png', 'dist': 101.3549417112076, 'mult': 1},
       {'name': 'lev4_88d732076134f7ea_rgb_1_.png', 'dist': 122.9247345327901, 'mult': 1},
       {'name': 'lev4_88d732015cca578c_rgb_6_.png', 'dist': 168.18839710766431, 'mult': 1},
       {'name': 'lev5_88d7320ddb1c7eb0_rgb_7_.png', 'dist': 280.9280404000206, 'mult': 1},
       {'name': 'lev5_88d7320cf5f45614_rgb_8_.png', 'dist': 236.62530147634214, 'mult': 1},
       {'name': 'lev4_88d7320624eac658_rgb_1_.png', 'dist': 115.25165277610472, 'mult': 1},
       {'name': 'lev3_88d731f382747df4_rgb_0_.png', 'dist': 74.10063978368407, 'mult': 1},
       {'name': 'lev5_88d732944311fea6_rgb_3_.png', 'dist': 222.00252570096768, 'mult': 1},
       {'name': 'lev6_88d7329b815c52e0_rgb_2_.png', 'dist': 436.8495070691849, 'mult': 1},
       {'name': 'lev6_88d7329e560f2b78_rgb_1_.png', 'dist': 481.76139568884696, 'mult': 1}],
      dtype=object)

In [20]:
%%bash
cp dataset/full/lev6_88d7329e560f2b78_rgb_1_.png dataset/sanity_check/

In [26]:
with open(os.path.join(SAVE_PATH, 'sanity_check', 'labels.json'), 'w+') as labels:
    labels = json.dump({'data': samples.tolist()}, labels)